In [60]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import load_model
from sklearn.externals import joblib
import re

In [8]:
pd.set_option('display.max_columns', None)

In [6]:
features = pd.read_csv('../datasets/features.csv')
features.rename(index=str, inplace=True, columns={
    'srcip': 'ipv4_src',
    'sport': 'port_src',
    'dstip': 'ipv4_dst',
    'dsport': 'port_dst'})
real = pd.read_csv('../datasets/UNSW-NB15_1.csv', names=features)
real[['ipv4_src', 'port_src', 'ipv4_dst', 'port_dst']] = real[['ipv4_src', 'port_src', 'ipv4_dst', 'port_dst']].astype(str)

/home/jozef/anaconda2/lib/python2.7/site-packages/pandas/io/parsers.py:678: UserWarning: Duplicate names specified. This will raise an error in the future.
  return _read(filepath_or_buffer, kwds)
/home/jozef/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (1,3,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [173]:
flows = pd.read_csv('../results/pcap6.dump', delimiter=',')
flows['real_label'] = 0

In [174]:
flows.shape

(80, 15)

In [175]:
packets_count = flows[['packets_src','packets_dst']].sum()
packets_count.sum()

1051

In [164]:
flows

,ipv4_src,port_src,ipv4_dst,port_dst,proto,bytes_src,bytes_dst,packets_src,packets_dst,srv_dst_count,dst_count,prediction,category,probability,real_label
0,59.166.0.0,38385,149.171.126.9,6881,6,132,140,2,2,4,5,[0],Normal,0.01%,0
1,59.166.0.7,35342,149.171.126.8,25,6,198,132,2,2,1,9,[0],Normal,0.13%,0
2,59.166.0.8,61926,149.171.126.3,6881,6,132,140,2,2,1,5,[0],Normal,0.17%,0
3,59.166.0.6,7018,149.171.126.7,6881,6,132,140,2,2,1,3,[0],Normal,0.23%,0
4,59.166.0.4,35902,149.171.126.8,6881,6,272,140,4,2,2,9,[0],Normal,5.47%,0
5,59.166.0.7,32578,149.171.126.5,6881,6,406,1356,6,12,6,8,[0],Normal,0.00%,0
6,59.166.0.7,47707,149.171.126.5,6881,6,406,1356,6,12,6,8,[0],Normal,0.00%,0
7,59.166.0.7,46481,149.171.126.5,6881,6,406,1356,6,12,6,8,[0],Normal,0.00%,0
8,59.166.0.7,3326,149.171.126.5,6881,6,1092,1356,8,12,6,8,[0],Normal,0.00%,0
9,59.166.0.7,4156,149.171.126.5,6881,6,1224,1356,10,12,6,8,[0],Normal,0.00%,0


In [165]:
def get_real_labels(ipv4_src, port_src, ipv4_dst, port_dst):
#     print ipv4_src, port_src, ipv4_dst, port_dst
    tmp = real['Label'].loc[(real['ipv4_src'] == ipv4_src) & 
               (real['port_src'] == port_src) &
               (real['ipv4_dst'] == ipv4_dst) &
               (real['port_dst'] == port_dst)]
#     print 'First:', tmp.values
    if tmp.values.size == 0:
        tmp = real['Label'].loc[(real['ipv4_src'] == ipv4_dst) & 
               (real['port_src'] == port_dst) &
               (real['ipv4_dst'] == ipv4_src) &
               (real['port_dst'] == port_src)]
#         print 'Second:', tmp.values
    return tmp.values[0]

flows[['ipv4_src', 'port_src', 'ipv4_dst', 'port_dst']] = flows[['ipv4_src', 'port_src', 'ipv4_dst', 'port_dst']].astype(str)
flows['real_label'] = flows.apply(lambda row: get_real_labels(row['ipv4_src'], row['port_src'], row['ipv4_dst'], row['port_dst']), axis=1)
flows.head()

,ipv4_src,port_src,ipv4_dst,port_dst,proto,bytes_src,bytes_dst,packets_src,packets_dst,srv_dst_count,dst_count,prediction,category,probability,real_label
0,59.166.0.0,38385,149.171.126.9,6881,6,132,140,2,2,4,5,[0],Normal,0.01%,0
1,59.166.0.7,35342,149.171.126.8,25,6,198,132,2,2,1,9,[0],Normal,0.13%,0
2,59.166.0.8,61926,149.171.126.3,6881,6,132,140,2,2,1,5,[0],Normal,0.17%,0
3,59.166.0.6,7018,149.171.126.7,6881,6,132,140,2,2,1,3,[0],Normal,0.23%,0
4,59.166.0.4,35902,149.171.126.8,6881,6,272,140,4,2,2,9,[0],Normal,5.47%,0


In [166]:
gr_data = flows[['ipv4_src', 'category']]
gr_data['count'] = 1
gr_data = gr_data.groupby(['ipv4_src', 'category']).count()
gr_data

/home/jozef/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,,count
ipv4_src,category,
149.171.126.1,Normal,2
149.171.126.12,Normal,1
149.171.126.5,Normal,1
149.171.126.7,Normal,1
149.171.126.9,Normal,1
59.166.0.0,Normal,11
59.166.0.2,Normal,8
59.166.0.3,Normal,7
59.166.0.4,Normal,5


In [167]:
evaluation = flows[['ipv4_src', 'category', 'real_label']]
evaluation['ipv4_src'] = evaluation.apply(lambda row: 'Malicious user' if row['real_label'] == 1 else 'Normal user', axis=1)
evaluation = evaluation[['ipv4_src', 'category']]
evaluation['count'] = 1
# evaluation = evaluation.replace(to_replace=r'(^59.*$)', value='Normal user', regex=True)
# evaluation = evaluation.replace(to_replace=r'(^175.*$)', value='Malicious user', regex=True)
evaluation = evaluation.groupby(['ipv4_src', 'category']).count()
evaluation

/home/jozef/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


count
ipv4_src       category       
Malicious user Normal        1
Normal user    Normal       76
               Warning       3

In [168]:
e = evaluation.unstack(fill_value=0)
e

count        
category       Normal Warning
ipv4_src                     
Malicious user      1       0
Normal user        76       3

In [169]:
try:
    TP = e.loc['Malicious user'].loc['count'].loc['Attack']
except:
    TP = 0
try:
    TN = e.loc['Normal user'].loc['count'].loc['Normal']
except:
    TN = 0
try:
    FP = e.loc['Normal user'].loc['count'].loc['Attack']
except:
    FP = 0
try:
    FN = e.loc['Malicious user'].loc['count'].loc['Normal']
except:
    FN = 0
print 'TP:', TP
print 'TN:', TN
print 'FP:', FP
print 'FN:', FN

TP: 0
TN: 76
FP: 0
FN: 1


In [170]:
try:
    acc = float(TP + TN)/float(TP + TN + FP + FN)
except:
    acc = 0
print ('Accuracy is %.2f%%' % (acc*100))
try:
    sen = float(TP)/float(TP + FN)
except:
    sen = 0
print ('Sensitivity is %.2f%%' % (sen*100))
try:
    spe = float(TN)/float(TN + FP)
except:
    spe = 0
print ('Specificity is %.2f%%' % (spe*100))
try:
    pre = float(TP)/float(TP + FP)
except:
    pre = 0
print ('Precision is %.2f%%' % (pre*100))
try:
    f1_score = 2*(float(pre*sen)/float(pre+sen))
except:
    f1_score = 0
print ('F1 score is %.2f%%' % (f1_score*100))

Accuracy is 98.70%
Sensitivity is 0.00%
Specificity is 100.00%
Precision is 0.00%
F1 score is 0.00%


In [172]:
flows.loc[flows['category'] == 'Warning']

,ipv4_src,port_src,ipv4_dst,port_dst,proto,bytes_src,bytes_dst,packets_src,packets_dst,srv_dst_count,dst_count,prediction,category,probability,real_label
13,59.166.0.6,11527,149.171.126.2,22437,6,878,736,10,8,1,5,[1],Warning,66.03%,0
27,59.166.0.4,40650,149.171.126.7,38455,6,1394,1142,16,14,1,3,[1],Warning,60.19%,0
32,59.166.0.9,42988,149.171.126.8,48017,6,1054,802,12,10,1,9,[1],Warning,67.02%,0


# Model evaluation, different test

In [58]:
model = load_model('../models/DNN_model_all_binary_full.h5')
scaler = joblib.load('../models/DNN_model_all_binary_full_scaler.sav')

/home/jozef/anaconda2/lib/python2.7/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator StandardScaler from version 0.20.3 when using version 0.20.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [150]:
def switch_values(flow):
    if re.match(r'^149.*', flow['ipv4_src']):
        tmp_ip = flow['ipv4_src']
        tmp_port = flow['port_src']
        tmp_bytes = flow['bytes_src']
        tmp_pkts = flow['packets_src']
        flow['ipv4_src'] = flow['ipv4_dst']
        flow['port_src'] = flow['port_dst']
        flow['ipv4_dst'] = tmp_ip
        flow['port_dst'] = tmp_port
        flow['bytes_src'] = flow['bytes_dst']
        flow['packets_src'] = flow['packets_dst']
        flow['bytes_dst'] = tmp_bytes
        flow['packets_dst'] = tmp_pkts
    return flow            

flow = flows.loc[flows['real_label'] == 1]
# flow = flow.apply(lambda x: switch_values(x), axis=1)
columns = flow.loc[:, 'packets_src':'packets_dst']
columns = pd.concat([columns, flow.loc[:, 'bytes_src':'bytes_dst']], axis=1)
columns = pd.concat([columns, flow.loc[:, 'srv_dst_count':'dst_count']], axis=1)
columns['proto_arp'] = columns['proto_icmp'] = columns['proto_igmp'] = columns['proto_ospf'] = columns['proto_other'] = columns['proto_tcp'] = columns['proto_udp'] = 0
for i in range(len(columns.index)):
    if flow.loc[columns.index[i]]['proto'] == -1:
        columns.loc[columns.index[i]]['proto_arp'] = 1
    elif flow.loc[columns.index[i]]['proto'] == 1:
        columns.loc[columns.index[i]]['proto_icmp'] = 1
    elif flow.loc[columns.index[i]]['proto'] == 2:
        columns.loc[columns.index[i]]['proto_igmp'] = 1
    elif flow.loc[columns.index[i]]['proto'] == 89:
        columns.loc[columns.index[i]]['proto_ospf'] = 1
    elif flow.loc[columns.index[i]]['proto'] == 6:
        columns.loc[columns.index[i]]['proto_tcp'] = 1
    elif flow.loc[columns.index[i]]['proto'] == 17:
        columns.loc[columns.index[i]]['proto_udp'] = 1
    else:
        columns.loc[columns.index[i]]['proto_other'] = 1
columns['packets_src'] = 10
columns['packets_dst'] = 10
columns['bytes_src'] = 842
columns

,packets_src,packets_dst,bytes_src,bytes_dst,srv_dst_count,dst_count,proto_arp,proto_icmp,proto_igmp,proto_ospf,proto_other,proto_tcp,proto_udp
78,10,10,842,240,1,1,0,0,0,0,0,1,0


In [151]:
model.predict_proba(scaler.transform(columns))

/home/jozef/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  """Entry point for launching an IPython kernel.


array([[0.76853204]], dtype=float32)

In [107]:
test = real.loc[(real['ipv4_src'] == flow.loc[78]['ipv4_src']) & 
               (real['port_src'] == flow.loc[78]['port_src']) &
               (real['ipv4_dst'] == flow.loc[78]['ipv4_dst']) &
               (real['port_dst'] == flow.loc[78]['port_dst'])]

In [112]:
test

,ipv4_src,port_src,ipv4_dst,port_dst,proto,state,dur,sbytes,dbytes,sttl,dttl,sloss,dloss,service,Sload,Dload,Spkts,Dpkts,swin,dwin,stcpb,dtcpb,smeansz,dmeansz,trans_depth,res_bdy_len,Sjit,Djit,Stime,Ltime,Sintpkt,Dintpkt,tcprtt,synack,ackdat,is_sm_ips_ports,ct_state_ttl,ct_flw_http_mthd,is_ftp_login,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat,Label
365,175.45.176.1,13377,149.171.126.12,80,tcp,FIN,0.187885,842,5084,62,252,2,3,http,32275.0625,194842.5938,10,10,255,255,114736493,263943961,84,508,1,2076,1103.263956,1668.959802,1421927443,1421927443,20.876111,19.461111,0.054718,0.009946,0.044772,0,1,1,0,0,1,1,1,1,1,1,1,Exploits,1


In [120]:
sample = test.loc[:, 'Spkts':'Dpkts']
sample = pd.concat([sample, test.loc[:, 'sbytes':'dbytes']], axis=1)
sample = pd.concat([sample, test.loc[:, 'ct_srv_dst':'ct_dst_ltm']], axis=1)
sample['proto_arp'] = sample['proto_icmp'] = sample['proto_igmp'] = sample['proto_ospf'] = sample['proto_other'] = sample['proto_tcp'] = sample['proto_udp'] = 0
i = 365
if test.loc[i]['proto'] == 'arp':
    sample['proto_arp'] = 1
elif test.loc[i]['proto'] == 'icmp':
    sample['proto_icmp'] = 1
elif test.loc[i]['proto'] == 'igmp':
    sample['proto_igmp'] = 1
elif test.loc[i]['proto'] == 'ospf':
    sample['proto_ospf'] = 1
elif test.loc[i]['proto'] == 'tcp':
    sample['proto_tcp'] = 1
elif test.loc[i]['proto'] == 'udp':
    sample['proto_udp'] = 1
else:
    sample['proto_other'] = 1
print (model.predict_proba(scaler.transform(sample)))

[[0.7120315]]


/home/jozef/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:20: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.


In [128]:
columns

,packets_src,packets_dst,bytes_src,bytes_dst,srv_dst_count,dst_count,proto_arp,proto_icmp,proto_igmp,proto_ospf,proto_other,proto_tcp,proto_udp
78,4,8,842,5112,1,1,0,0,0,0,0,1,0


In [129]:
sample

,Spkts,Dpkts,sbytes,dbytes,ct_srv_dst,ct_dst_ltm,proto_arp,proto_icmp,proto_igmp,proto_ospf,proto_other,proto_tcp,proto_udp
365,10,10,842,5084,1,1,0,0,0,0,0,1,0


In [191]:
df = pd.DataFrame([[acc, sen, spe, pre]], columns=means_names)
means = means.append(df)

In [192]:
means

,Accuracy,Sensitivity,Specificity,Precision
0,0.750000,1.000000,0.727273,0.250000
0,0.807692,1.000000,0.791667,0.285714
0,0.761905,0.666667,0.777778,0.333333
0,0.741935,0.428571,0.833333,0.428571
0,0.769231,0.800000,0.761905,0.444444
0,0.780000,0.750000,0.782609,0.230769
0,0.702128,0.600000,0.714286,0.200000
0,0.977273,0.888889,0.983740,0.800000
0,0.975410,1.000000,0.973684,0.727273


In [193]:
means['Accuracy'].mean()

0.8072859495123974

In [194]:
means['Sensitivity'].mean()

0.7926807760141092

In [195]:
means['Specificity'].mean()

0.8162526360908716

In [196]:
means['Precision'].mean()

0.4111228277894945

## Notes and unused commands

In [6]:
means_names = ['Accuracy','Sensitivity','Specificity','Precision']
means = pd.DataFrame(columns=means_names)
means

,Accuracy,Sensitivity,Specificity,Precision
